In [ ]:
pip install transformers

In [ ]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer


# Step 1: Download the JSON file
url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
response = requests.get(url)
json_data = response.json()

def json_to_dataframe(json_data):
    data = []
    for document in json_data:
        doc_id = document.get("id")
        for annotation in document.get("annotations", []):
            for result in annotation.get("result", []):
                segment = {
                    'doc_id': doc_id,
                    'text': result['value'].get('text'),
                    'label': result['value'].get('labels', [None])[0]  # Get the first label if available
                }
                data.append(segment)
    return pd.DataFrame(data)

# Convert JSON to DataFrame
df = json_to_dataframe(json_data)

In [ ]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

# Convert DataFrame to Hugging Face Dataset again
dataset = Dataset.from_pandas(df)

{'ANALYSIS': 0, 'ARG_PETITIONER': 1, 'ARG_RESPONDENT': 2, 'FAC': 3, 'ISSUE': 4, 'NONE': 5, 'PREAMBLE': 6, 'PRE_NOT_RELIED': 7, 'PRE_RELIED': 8, 'RATIO': 9, 'RLC': 10, 'RPC': 11, 'STA': 12}


In [ ]:
# Load the tokenizer and model for ERNIE 2.0
model_name = "nghuyong/ernie-2.0-base-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(df['label'])))

# Step 2: Preprocess function for tokenization
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",  # Pads sequences to the model's max length
        truncation=True,  # Truncates sequences longer than the model's max length
        max_length=100,  # Adjust this value to the model's expected input length, if necessary
    )
# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

from datasets import DatasetDict
# Split the dataset into train, test, and eval sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_test_dataset = split_dataset['test'].train_test_split(test_size=0.5)

datasets = DatasetDict({
    'train': split_dataset['train'],
    'test': train_test_dataset['test'],
    'eval': train_test_dataset['train']
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/28986 [00:00<?, ? examples/s]

In [ ]:
# Step 3: Setup Training Arguments
training_args = TrainingArguments(
    output_dir="./resultsERNIE",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_total_limit=1,
    save_steps=500,
    logging_dir="./logs",
    report_to="wandb"  # Ensure W&B is properly set up, or remove this line
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['eval']
)

# Step 4: Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


ValueError: You are trying to save a non contiguous tensor: `ernie.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# If the model is already trained, just initialize the Trainer with the same model
trainer = Trainer(
    model=model,  # Use your already trained model
    args=training_args,
    eval_dataset=datasets['eval'],  # Use your evaluation dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results
print(results)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Initialize the Trainer with the already trained model
trainer = Trainer(
    model=model,  # Use your already trained model
    #args=training_args,
    eval_dataset=datasets['test'],  # Use your test dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print the evaluation results
print("Test Set Evaluation Results:")
print(results)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 19


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Test Set Evaluation Results:
{'eval_loss': 2.4910380840301514, 'eval_model_preparation_time': 0.0055, 'eval_accuracy': 0.21076233183856502, 'eval_precision': 0.1763085437869524, 'eval_recall': 0.21076233183856502, 'eval_f1': 0.17898957089964976, 'eval_runtime': 31.7021, 'eval_samples_per_second': 91.445, 'eval_steps_per_second': 11.45}


# save this model on huggingface

In [ ]:
pip install huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from huggingface_hub import HfApi

# Define repository name and create repo on the hub
repo_name = "ERNIE2.0_for__for_rhetorical_role_labeling"  # Choose a unique name
HfApi().create_repo(repo_name)

# Upload the model
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/engineersaloni159/ERNIE2.0_for__for_rhetorical_role_labeling/commit/a67bc90256659b36fc037225727bd25efcb4c9db', commit_message='Upload tokenizer', commit_description='', oid='a67bc90256659b36fc037225727bd25efcb4c9db', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained(repo_name, push_to_hub=True)
tokenizer.save_pretrained(repo_name, push_to_hub=True)


('ERNIE2.0_for__for_rhetorical_role_labeling/tokenizer_config.json',
 'ERNIE2.0_for__for_rhetorical_role_labeling/special_tokens_map.json',
 'ERNIE2.0_for__for_rhetorical_role_labeling/vocab.txt',
 'ERNIE2.0_for__for_rhetorical_role_labeling/added_tokens.json',
 'ERNIE2.0_for__for_rhetorical_role_labeling/tokenizer.json')

# Manually Evaluation of model

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("engineersaloni159/DistillBERT_for_rhetorical_rolelabeling")
model = AutoModelForSequenceClassification.from_pretrained("engineersaloni159/DistillBERT_for_rhetorical_rolelabeling")


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Ensure model is in evaluation mode
model.eval()

In [ ]:
# Assuming label_encoder is the one you used for encoding labels earlier

# Example sentence
sentence = "The State is aggrieved against an order passed by the High Court of Judicature at Madras on 24th April, 2019 constituting a Heterogeneous Committee of named persons to give its recommendations on the reforms that can be brought into practice for reformation, rehabilitation and re-integration of the convict/accused person to society and best practices for improving the quality of investigation."
# Tokenize the sentence
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

# The predictions are in the form of logits, you can convert them to probabilities
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1)

# Convert the predicted class number to the original label
predicted_label = label_encoder.inverse_transform(predicted_class.cpu().numpy())[0]

# Print the predicted label
print(f"Predicted label: {predicted_label}")


Predicted label: FAC


labelwise score

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=range(len(label_mapping)))
    accuracy = accuracy_score(labels, preds)

    metrics = {}
    for i, label in enumerate(label_mapping):
        metrics[f"precision_{label}"] = precision[i]
        metrics[f"recall_{label}"] = recall[i]
        metrics[f"f1_{label}"] = f1[i]

    metrics["accuracy"] = accuracy
    return metrics
eval_dataset = datasets['eval']

In [ ]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results",
        per_device_eval_batch_size=8,  # Adjust batch size as needed
        logging_dir="./logs",
        report_to="none"  # Set to 'wandb' if using Weights & Biases
    ),
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate(eval_dataset)
print(eval_results)


{'eval_loss': 0.6045954823493958, 'eval_precision_ANALYSIS': 0.7796887796887797, 'eval_recall_ANALYSIS': 0.8972667295004713, 'eval_f1_ANALYSIS': 0.8343558282208589, 'eval_precision_ARG_PETITIONER': 0.6567164179104478, 'eval_recall_ARG_PETITIONER': 0.6376811594202898, 'eval_f1_ARG_PETITIONER': 0.6470588235294118, 'eval_precision_ARG_RESPONDENT': 0.7857142857142857, 'eval_recall_ARG_RESPONDENT': 0.1527777777777778, 'eval_f1_ARG_RESPONDENT': 0.2558139534883721, 'eval_precision_FAC': 0.8648180242634316, 'eval_recall_FAC': 0.8588640275387264, 'eval_f1_FAC': 0.8618307426597582, 'eval_precision_ISSUE': 0.8636363636363636, 'eval_recall_ISSUE': 0.8837209302325582, 'eval_f1_ISSUE': 0.8735632183908046, 'eval_precision_NONE': 0.9361702127659575, 'eval_recall_NONE': 0.8979591836734694, 'eval_f1_NONE': 0.9166666666666666, 'eval_precision_PREAMBLE': 0.8865435356200527, 'eval_recall_PREAMBLE': 0.8571428571428571, 'eval_f1_PREAMBLE': 0.8715953307392996, 'eval_precision_PRE_NOT_RELIED': 0.0, 'eval_recal

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
